In [1]:
# 현재 파일 위치 확인
import os
print(os.getcwd())

C:\Users\TitusChoi\Desktop\Library\A2W_Project


In [2]:
import pandas as pd
import csv

In [3]:
# Merged_data_file
Merged_2020 = pd.read_csv('datasets/merge_s_h_a20.csv')
Merged_2020.head()

,tm,stnId,stnNm,umd,sgg,umd2,avgTa,minTa,maxTa,mi10MaxRn,...,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
0,2020-01-01,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,2.1,-0.6,3.8,0.0,...,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
1,2020-01-02,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,4.7,2.7,7.2,0.0,...,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
2,2020-01-03,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,5.9,4.1,8.5,0.0,...,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
3,2020-01-04,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,5.0,2.7,8.1,0.0,...,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0
4,2020-01-05,115,울릉도,울릉읍,울릉군,울릉군 울릉읍,4.4,2.7,6.2,0.0,...,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0


In [4]:
# Test 검증 데이터셋
test_input = pd.read_csv('datasets/input.csv', encoding = 'cp949')
test_input.head()

,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,NaN,2020-06-12,NaN
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,NaN,2020-06-13,NaN
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,NaN,2020-06-14,NaN
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,NaN,2020-06-12,NaN
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,NaN,2020-06-13,NaN


In [6]:
# searching data 찾아야 됨
def search_data(sgg, umd, date):
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    specific = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    return specific

'''
input 예시
input_date = '2020-06-11' # 날짜 단위
input_sgg = '울릉군' # 시 단위
input_umd = '울릉읍' # 읍면동 단위
'''

"\ninput 예시\ninput_date = '2020-06-11' # 날짜 단위\ninput_sgg = '울릉군' # 시 단위\ninput_umd = '울릉읍' # 읍면동 단위\n"

In [5]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [6]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

In [7]:
raw_df = pd.read_csv("datasets/merge_final_real.csv", encoding='utf-8-sig')

In [8]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

In [9]:
raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]

c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)
cleaned_df_1.head()

,maxTa,minTa,avgTa,EIGHT_AGL,stnId,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,...,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,1day_yn
0,2.1,-1.1,0.8,0.0,115,0.0,0.0,19.33,28.0,29.4,...,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,1.3,-0.9,0.1,0.0,115,0.0,0.0,22.83,24.8,14.5,...,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,0.8,-1.4,-0.3,0.0,115,0.0,0.0,12.83,7.5,15.9,...,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,3.9,0.4,1.7,0.0,115,0.0,0.0,0.00,0.0,11.2,...,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,2.6,-1.2,1.2,0.0,115,0.0,0.0,7.25,1.8,13.9,...,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0


In [11]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)

In [12]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, sampling_strategy={1: 359, 0: 17950}).fit_resample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (13731, 33)
X_test shape: (4578, 33)
y_train shape: (13731, 1)
y_test shape: (4578, 1)


In [14]:
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.2, max_depth = 6)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 10000, eval_metric = "logloss", eval_set = evals, verbose = 1)

c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[0]	validation_0-logloss:0.52475
[1]	validation_0-logloss:0.40943
[2]	validation_0-logloss:0.32727
[3]	validation_0-logloss:0.26595
[4]	validation_0-logloss:0.21964
[5]	validation_0-logloss:0.18371
[6]	validation_0-logloss:0.15523
[7]	validation_0-logloss:0.13206
[8]	validation_0-logloss:0.11394
[9]	validation_0-logloss:0.09943
[10]	validation_0-logloss:0.08712
[11]	validation_0-logloss:0.07723
[12]	validation_0-logloss:0.06899
[13]	validation_0-logloss:0.06242
[14]	validation_0-logloss:0.05729
[15]	validation_0-logloss:0.05272
[16]	validation_0-logloss:0.04879
[17]	validation_0-logloss:0.04537
[18]	validation_0-logloss:0.04267
[19]	validation_0-logloss:0.03935
[20]	validation_0-logloss:0.03688
[21]	validation_0-logloss:0.03462
[22]	validation_0-logloss:0.03358
[23]	validation_0-logloss:0.03253
[24]	validation_0-logloss:0.03159
[25]	validation_0-logloss:0.03017
[26]	validation_0-logloss:0.02947
[27]	validation_0-logloss:0.02873
[28]	validation_0-logloss:0.02803
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
xgb_pred = xgb.predict(X_test)
metrics(y_test, xgb_pred)

정확도 : 1.00, 정밀도 : 0.95, 재현율 : 0.80
f1-score : 0.87, auc : 0.90


In [16]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','2day_yn']]
cleaned_df_2 = select_df.fillna(0)

In [17]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)

In [18]:
# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, sampling_strategy={1: 359, 0: 17950}).fit_resample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (13731, 33)
X_test shape: (4578, 33)
y_train shape: (13731, 1)
y_test shape: (4578, 1)


In [20]:
# 최고 하이퍼 파라미터 값으로 예측 실시
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.2, max_depth = 6)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

[0]	validation_0-logloss:0.52363
[1]	validation_0-logloss:0.40994
[2]	validation_0-logloss:0.32821
[3]	validation_0-logloss:0.26602
[4]	validation_0-logloss:0.21908
[5]	validation_0-logloss:0.18314
[6]	validation_0-logloss:0.15394
[7]	validation_0-logloss:0.13112
[8]	validation_0-logloss:0.11317
[9]	validation_0-logloss:0.09904
[10]	validation_0-logloss:0.08711


c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[11]	validation_0-logloss:0.07789
[12]	validation_0-logloss:0.07042
[13]	validation_0-logloss:0.06411
[14]	validation_0-logloss:0.05735
[15]	validation_0-logloss:0.05313
[16]	validation_0-logloss:0.04917
[17]	validation_0-logloss:0.04604
[18]	validation_0-logloss:0.04348
[19]	validation_0-logloss:0.04179
[20]	validation_0-logloss:0.03915
[21]	validation_0-logloss:0.03791
[22]	validation_0-logloss:0.03695
[23]	validation_0-logloss:0.03564
[24]	validation_0-logloss:0.03499
[25]	validation_0-logloss:0.03436
[26]	validation_0-logloss:0.03337
[27]	validation_0-logloss:0.03289
[28]	validation_0-logloss:0.03173
[29]	validation_0-logloss:0.03137
[30]	validation_0-logloss:0.03011
[31]	validation_0-logloss:0.03015
[32]	validation_0-logloss:0.02943
[33]	validation_0-logloss:0.02917
[34]	validation_0-logloss:0.02843
[35]	validation_0-logloss:0.02829
[36]	validation_0-logloss:0.02823
[37]	validation_0-logloss:0.02821
[38]	validation_0-logloss:0.02836
[39]	validation_0-logloss:0.02814
[40]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
xgb_pred = xgb.predict(X_test)
metrics(y_test, xgb_pred)

정확도 : 0.99, 정밀도 : 0.93, 재현율 : 0.76
f1-score : 0.83, auc : 0.88


In [24]:
# 훈련된 모델로 결과 도출 : day1 함수
def day1_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day1 = test_specific_day1.fillna(0)
    xgb_pred = xgb.predict(fillna_test_specific_day1)
    return xgb_pred

# 훈련된 모델로 결과 도출 : day2 함수
def day2_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day2 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day2 = test_day2[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day2 = test_specific_day2.fillna(0)
    xgb_pred = xgb.predict(fillna_test_specific_day2)
    return xgb_pred

In [25]:
length_test_input = len(test_input.index)

#day_1 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['1day_yn'][index] = day1_data_test(input_sgg, input_umd, input_date)
        

#day_2 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['2day_yn'][index] = day2_data_test(input_sgg, input_umd, input_date)
        
# print(search_data(input_sgg, input_umd, input_date))


test_input.head()

c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\tituschoi\desktop\library\a2w_project\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,0.0,2020-06-12,0.0
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,0.0,2020-06-13,0.0
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,0.0,2020-06-14,0.0
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,0.0,2020-06-12,0.0
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,0.0,2020-06-13,0.0


In [27]:
test_input.to_csv("results/test_result.csv", index=False, encoding='utf-8-sig')